# A.2. GWR Analysis of Student Well-being and Academic Performance

This Python script conducts a GWR analysis to investigate the spatially varying relationships between
various predictors and student academic performance:

In [ ]:
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

# Load the dataset
file_path = 'C:\\Users\\home_folder\\students_wellbeing_digital.xlsx'
df = pd.read_excel(file_path)

# Parse seating coordinates
def parse_m1(m1_str):
    if pd.isna(m1_str):
        return None, None
    matrix_size, position = m1_str.split('=')
    cols, rows = map(int, matrix_size.split('x'))
    col, row = map(int, position.split('.'))
    return (col, row)

df['M1_col'], df['M1_row'] = zip(*df['M1'].apply(parse_m1))
df['M1_col'].fillna(99, inplace=True)
df['M1_row'].fillna(99, inplace=True)

# Prepare data
y = df['2G'].values.reshape((-1, 1))
X = df[['1P1','1P2','1P3','1P4','1P5','2P1','2P2','2P3','2P4','2P5']].values
coords = df[['M1_col', 'M1_row']].values

# Impute and scale predictors
X = SimpleImputer(strategy='mean').fit_transform(X)
X = StandardScaler().fit_transform(X)

# Select bandwidth and fit GWR
bw = Sel_BW(coords, y, X).search()
gwr_model = GWR(coords, y, X, bw=bw)
results = gwr_model.fit()
params = results.params

# Plot results
for i, col in enumerate(['1P1','1P2','1P3','1P4','1P5','2P1','2P2','2P3','2P4','2P5']):
    plt.figure(figsize=(10,6))
    plt.scatter(coords[:,0], coords[:,1], c=params[:,i+1], cmap='coolwarm', edgecolor='k', s=80)
    plt.colorbar(label='Coefficient')
    plt.xlabel('Column')
    plt.ylabel('Row')
    plt.title(f'GWR Coefficients for {col}')
    plt.grid(True)
    plt.show()

# Print GWR summary
results.summary()